# Chest X-Ray Classification Analysis

# Imports

In [4]:
import os

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from PIL import Image, ImageFile

# Create Paths

In [2]:
PATH = "/Users/maukamir/Downloads/chest_xray/"
train_path, val_path, test_path = PATH + "train", PATH + "val", PATH + "test"

# Constants

In [3]:
BATCH_SIZE = 32
INPUT_SHAPE = (224,224,3)

# Functions

In [5]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

def process_load_image(img_path, target_size=INPUT_SHAPE):
  try:
    with Image.open(img_path) as img:
      img = img.resize(target_size)
      img = img.convert("RGB")
      
      img_array = np.array(img)/255.0
      img_array = img_array.astype(np.float32)
      return img_array
  except IOError as e:
    print(f"Failure in loading this image: {e}")
    return None


def grab_images(path):
  
  images, labels = [],[]
  
  for root, dir, files in os.walk(path):
    for file in files:
      img_path = os.path.join(root, file)
      img = process_load_image(img_path)
      if img is not None:
        images.append(img)
        label = os.sep(root)[:-1]
        labels.append(label)
  
  images, labels = np.array(images), np.array(labels)
  
  return images, labels



# Load and Extract Images